Импортируем модули и глобальные переменные на уровне модулей.


In [1]:
from src import data_utils, data_set, lstm_model, transformer

TOKENIZER = data_set.TOKENIZER
MIN_LEN = data_set.MIN_LEN
ROUGE = lstm_model.ROUGE

/home/vlabs/Documents/Yandex_Practicum_ML_CV/venv_sprint_2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Готовим данные:
- Загружаем датасет.
- Чистим его с помощью регулярных выражений.
- Разбиваем на сеты для обучения и валидации.

In [2]:
url = "https://code.s3.yandex.net/deep-learning/tweets.txt"
filename = "data/raw_dataset.txt"
data_utils.download_from_url(url, filename)

In [ ]:
# Аргумент cap ограничивает число строк, которые будут загружены.
# Значения до 100К удобны для проверки. Тетрадка отработает за несколько минут.
clean = data_utils.clean_up(data_utils.load_dataset(
    "data/raw_dataset.txt", cap=50000))

splits = data_utils.split_dataset(clean)

data_sets = []
data_loaders = []
for split in [splits['train'], splits['val']]:
    dset = data_set.TwitterDataset(split)
    data_sets.append(dset)
    data_loaders.append(dset.get_loader())

100%|██████████| 100000/100000 [00:00<00:00, 212228.62it/s]


train set len:       80000
validataion set len: 10000
test set len:        10000


100%|██████████| 10000/10000 [00:01<00:00, 9025.77it/s]


Обучаем LSTM модель

In [4]:
lstm = lstm_model.Lstm(TOKENIZER)
lstm.train_model(n_epochs=3, l_rate=0.002, tokenizer=TOKENIZER,
                 train_loader=data_loaders[0], val_loader=data_loaders[1])

# Сохраняем модель для инференса
lstm.save_state_dict("models/lstm_state_dict.pth")

# И целиком
lstm.save("models/lstm_entire.pth")

100%|██████████| 305/305 [00:08<00:00, 37.60it/s]


Epoch 1 | Train Loss: 6.857 | Val Loss: 6.104 | Val Accuracy: 15.89%
ROUGE metrics
rouge1: 0.2515
rouge2: 0.0177
rougeL: 0.1486
rougeLsum: 0.1489


100%|██████████| 305/305 [00:06<00:00, 44.15it/s]


Epoch 2 | Train Loss: 5.321 | Val Loss: 5.807 | Val Accuracy: 19.14%
ROUGE metrics
rouge1: 0.2938
rouge2: 0.0216
rougeL: 0.1603
rougeLsum: 0.1607


100%|██████████| 305/305 [00:07<00:00, 42.09it/s]


Epoch 3 | Train Loss: 4.459 | Val Loss: 5.913 | Val Accuracy: 20.03%
ROUGE metrics
rouge1: 0.2959
rouge2: 0.0221
rougeL: 0.1637
rougeLsum: 0.1639


В ходе обучения предсказываем 3 слова для каждого предложения.

Почему так ? В задании сказано "предсказать оставшуюся 1/4 часть предложения". \
Т. к. предложения разной длины, то элементы батча будут обработаны разное число раз. \
Это ограничит возможности использования батчей, вот возможные варианты:
- Отказаться от батчей вовсе.
- Отсортировать предложения по длине и использовать маленький батч (чтобы длины в батче совпадали)
- Зафиксирваоть число слов (1/4 минимальной длины предложения или максмальной и т. п.)


Получается логика обработки, которая не имеет никакого отношения к обучению нейросетям. Как будто, эта работа лишняя. \
Если вы настаиваете, я решу задачу одним из описанных способов.

In [5]:
NUM_WORDS = 3

Инференс с авторегрессией

In [6]:
# Заполняем датасет для инференса, т. к. мы обучаем модель предсказывать только
# 1 слово, а инференс хотим делать для нескольких.
lstm_data = []
true_preds = []
full_texts = []

for i in range(0, len(splits['test'])):
    words = splits['test'][i].split()
    if len(words) < MIN_LEN + NUM_WORDS:
        continue
    ctx_len = len(words) - NUM_WORDS
    context = ' '.join(words[:ctx_len])
    target = ' '.join(words[ctx_len:])
    lstm_data.append(context)
    true_preds.append(target)
    full_texts.append(context + " " + target)

# Данные для инференса трансформера будут точно такие же, делаем копию.
gpt2_data = lstm_data

In [7]:
# Здесь храним только предсказанные слова.
pred_data = [""] * len(lstm_data)

# Проходимся по всему датасету и генерируем массив предсказанных слов.
# Далее, прибавляем предсказанное слово ко входам.
for i in range(0, NUM_WORDS):
    dataset = data_set.TwitterDataset(lstm_data, shuffle=False, num_targets=0)
    preds = lstm.inference(loader=dataset.get_loader(), tokenizer=TOKENIZER)
    lstm_data = [x + ' ' + y for x, y in zip(lstm_data, preds)]
    pred_data = [x + ' ' + y for x, y in zip(pred_data, preds)]

rouge_score = ROUGE.compute(predictions=pred_data, references=true_preds)
print('ROUGE metrics')
for k, v in rouge_score.items():
    print(f"{k}: {v:.4f}")

100%|██████████| 8181/8181 [00:00<00:00, 14839.76it/s]


ROUGE metrics
rouge1: 0.0335
rouge2: 0.0020
rougeL: 0.0333
rougeLsum: 0.0333


Аналогичная процедура для трансформера.

In [8]:
pred_data = [""] * len(lstm_data)

gpt2 = transformer.DistilGPT2()
for i in range(0, NUM_WORDS):
    preds = gpt2.inference(gpt2_data)
    gpt2_data = [x + ' ' + y for x, y in zip(gpt2_data, preds)]
    pred_data = [x + ' ' + y for x, y in zip(pred_data, preds)]

rouge_score = ROUGE.compute(predictions=pred_data, references=true_preds)
print('ROUGE metrics')
for k, v in rouge_score.items():
    print(f"{k}: {v:.4f}")

Device set to use cuda:0
8181it [00:26, 306.10it/s]                    
8181it [00:28, 290.74it/s]                    
8181it [00:33, 243.94it/s]                    


ROUGE metrics
rouge1: 0.0630
rouge2: 0.0088
rougeL: 0.0627
rougeLsum: 0.0627


Посмотрим на некоторые результаты работы

In [9]:
print("\nSome LSTM predictions:")
for i in range(0, 3):
    print(lstm_data[i])

print("\nSome GPT2 predictions:")
for i in range(0, 3):
    print(gpt2_data[i])

print("\nActual sentences:")
for i in range(0, 3):
    print(full_texts[i])


Some LSTM predictions:
nicksantino warped but you guys wont be on the florida dates ##s ##nl ##r
miafreedman i m loving harem pants at the moment seriously but not the ones with a crutch to twitter it yet
watched some more planet earth with my dad the ice worlds episode has super sad moments poor polar bears ##y ##y work

Some GPT2 predictions:
nicksantino warped but you guys wont be on the florida dates and are so
miafreedman i m loving harem pants at the moment seriously but not the ones with a crutch to grab and grab
watched some more planet earth with my dad the ice worlds episode has super sad moments poor polar bears in a forest

Actual sentences:
nicksantino warped but you guys wont be on the florida dates im so sad
miafreedman i m loving harem pants at the moment seriously but not the ones with a crutch to the knee ewwww
watched some more planet earth with my dad the ice worlds episode has super sad moments poor polar bears and baby penguins


Выводы:
- Использование трансформера избыточно для этой задачи.
- LSTM с небольшим скрытым состоянием (128) справляется с задачей предсказания следующего слова лучше.
- Эксперименты по увеличению размера скрытого состояния LSTM (до 384) показали, что на коротких текстах это бессмысленно. Accuracy достигает значения, близкого к максимальному уже на 1й эпохе, а потом прекращает рост.
- Меньший размер скрытого состояния (128) даёт схожее значение accuracy уже на 3й эпохе обучения.
- Траснформер выдаёт более "осмысленные", но менее точные предсказания при авторегрессии.